In [0]:
%spark.pyspark
import h5py
import tables
import os
import sys
sys.path.insert(0, "/home/hadoop/10605_Group8")
from hdf5_getters import *
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.clustering import KMeans
from pyspark.ml.evaluation import ClusteringEvaluator
from pyspark.ml import Pipeline

In [1]:
%spark.pyspark
def getListOfFiles(dirName):
    # create a list of file and sub directories 
    # names in the given directory 
    listOfFile = os.listdir(dirName)
    allFiles = list()
    # Iterate over all the entries
    for entry in listOfFile:
        # Create full path
        fullPath = os.path.join(dirName, entry)
        # If entry is a directory then get the list of files in this directory 
        if os.path.isdir(fullPath):
            allFiles = allFiles + getListOfFiles(fullPath)
        else:
            allFiles.append(fullPath)
                
    return allFiles

# TODO: add more attribute
def read_h5_to_list(filename):
    import sys
    import h5py
    import tables
    sys.path.insert(0, "/home/hadoop/10605_Group8")
    from hdf5_getters import *
    h5tocopy = open_h5_file_read(filename)
    song_num = get_num_songs(h5tocopy)
    result = []
    for songidx in range(song_num):
        song_info = []
        # METADATA
        song_info.append(float(get_artist_familiarity(h5tocopy,songidx)))
        song_info.append(float(get_artist_hotttnesss(h5tocopy,songidx)))
        song_info.append(str(get_artist_id(h5tocopy,songidx)))
        song_info.append(str(get_artist_location(h5tocopy,songidx)))
        song_info.append(get_artist_mbtags(h5tocopy,songidx).tolist())
        song_info.append(get_artist_mbtags_count(h5tocopy,songidx).tolist())
        song_info.append(str(get_artist_name(h5tocopy,songidx)))
        song_info.append(get_artist_terms(h5tocopy,songidx).tolist())
        song_info.append(get_artist_terms_freq(h5tocopy,songidx).tolist())
        song_info.append(get_artist_terms_weight(h5tocopy,songidx).tolist())
        song_info.append(float(get_danceability(h5tocopy,songidx)))
        song_info.append(float(get_duration(h5tocopy,songidx)))
        song_info.append(float(get_end_of_fade_in(h5tocopy,songidx)))
        song_info.append(float(get_energy(h5tocopy,songidx)))
        song_info.append(float(get_key(h5tocopy,songidx)))
        song_info.append(float(get_key_confidence(h5tocopy,songidx)))
        song_info.append(float(get_loudness(h5tocopy,songidx)))
        song_info.append(float(get_mode(h5tocopy,songidx)))
        song_info.append(float(get_mode_confidence(h5tocopy,songidx)))
        song_info.append(str(get_release(h5tocopy,songidx)))
        song_info.append(get_segments_confidence(h5tocopy,songidx).tolist())        
        song_info.append(get_segments_loudness_max(h5tocopy,songidx).tolist())        
        song_info.append(get_segments_loudness_max_time(h5tocopy,songidx).tolist())    
        song_info.append(get_segments_pitches(h5tocopy,songidx).tolist())    
        song_info.append(get_segments_timbre(h5tocopy,songidx).tolist())    
        song_info.append(get_similar_artists(h5tocopy,songidx).tolist())   
        song_info.append(float(get_artist_hotttnesss(h5tocopy,songidx)))
        song_info.append(str(get_song_id(h5tocopy,songidx)))
        song_info.append(float(get_start_of_fade_out(h5tocopy,songidx)))
        song_info.append(float(get_tempo(h5tocopy,songidx)))
        song_info.append(int(get_time_signature(h5tocopy,songidx)))
        song_info.append(float(get_time_signature_confidence(h5tocopy,songidx)))
        song_info.append(str(get_title(h5tocopy,songidx)))
        song_info.append(str(get_track_id(h5tocopy,songidx)))
        song_info.append(int(get_year(h5tocopy,songidx)))

        result.append(song_info)
    h5tocopy.close()
    return result

In [2]:
%spark.pyspark
sc._jsc.hadoopConfiguration().set("fs.s3a.access.key", ACCESS_KEY)
sc._jsc.hadoopConfiguration().set("fs.s3a.secret.key", SECRET_KEY)
sc._jsc.hadoopConfiguration().set("fs.s3a.endpoint", "s3.amazonaws.com")

In [3]:
%spark.pyspark
    
num_nodes = 2*4*4

data_path = '/mnt/snap/data'
filenames = getListOfFiles(data_path)
rdd = sc.parallelize(filenames, num_nodes)
rdd1 = rdd.flatMap(lambda x: read_h5_to_list(x))
col_name = ["artist_familiarity", "artist_hotttnesss", "artist_id", "artist_location", "artist_mbtags", 
 "artist_mbtags_count", "artist_name", "artist_terms", "artist_terms_freq", "artist_terms_weight", 
 "danceability", "duration", "end_of_fade_in", "energy", "key",
"key_confidence", "loudness", "mode", "mode_confidence", "release", 
 "segments_confidence", "segments_loudness_max", "segments_loudness_max_time", 
"segments_pitches", "segments_timbre", "similar_artists", 
"song_hotttnesss", "song_id", "start_of_fade_out", "tempo", "time_signature", 
"time_signature_confidence", "title", "track_id", "year"]

df = rdd1.toDF(col_name)


In [5]:
sc.hadoopConfiguration.set("fs.s3a.access.key", ACCESS_KEY)
sc.hadoopConfiguration.set("fs.s3a.secret.key", SECRET_KEY)
sc.hadoopConfiguration.set("fs.s3a.endpoint", "s3.amazonaws.com")
df.write.mode("overwrite").parquet("s3a://msdbucket/parquet")


In [7]:
%spark.pyspark
from pyspark.sql import SQLContext
sqlContext = SQLContext(sc)
dfParquet = sqlContex.read.parquet("s3a://msdbucket/parquet")